In [8]:
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from llama_index import LLMPredictor, PromptHelper, SimpleDirectoryReader, GPTSimpleVectorIndex, Document, GPTListIndex
from langchain import OpenAI
from langchain.vectorstores import  Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from time import sleep
from tqdm import tqdm
import pinecone
import uuid
import json
import openai

embed_model = "text-embedding-ada-002"

In [3]:
import pinecone
index_name = 'game-reviews'

# initialize connection to pinecone (get API key at app.pinecone.io)
pinecone.init(
    api_key="b32bb33f-57b0-47f5-8669-949c9f6f2f4b",
    environment="us-west1-gcp"  # may be different, check at app.pinecone.io
)
# connect to index
index = pinecone.Index(index_name)
# view index stats
index.describe_index_stats()


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 317}},
 'total_vector_count': 317}

In [ ]:
# 拉取YouTube上的transcript
def loadYoutubeData(url):
    loader = YoutubeLoader.from_youtube_url(
        url, add_video_info=True)
    data = loader.load()
    # save_file('data/%s.json' % data[0].metadata['title'], data[0].page_content)

    return data


In [ ]:
# video_url= "https://www.youtube.com/watch?v=ziFipUmd7Mg"#My Time At Portia
# video_url = "https://www.youtube.com/watch?v=V56uazDSYcA" #Animal Crossing:
# video_url = "https://www.youtube.com/watch?v=hI99HvOHCLI" #Star valley
# video_url = "https://www.youtube.com/watch?v=zDE9UbiY7fg"
# video_url = "https://www.youtube.com/watch?v=87MSamVRHxU"
# video_url = "https://www.youtube.com/watch?v=ewYXxWujytA"
# video_url = "https://www.youtube.com/watch?v=WNk6CGJdnIA"
# video_url = "https://www.youtube.com/watch?v=Ogn5DcRIcbs"
# video_url = "https://www.youtube.com/watch?v=q6rBB14SHNg"
# video_url = "https://www.youtube.com/watch?v=Sr5uA7TSAwI"
# video_url = "https://www.youtube.com/watch?v=NBlTyGPgKiA"
# video_url = "https://www.youtube.com/watch?v=zlSq83r3y5s"
# video_url = "https://www.youtube.com/watch?v=LM7zZgtf92Y"
# video_url = "https://www.youtube.com/watch?v=9xj4ciP0Riw"
# video_url = "https://www.youtube.com/watch?v=bJyyPXV1yV8"
# video_url = "https://www.youtube.com/watch?v=laIXYT9cdCQ"
# video_url = "https://www.youtube.com/watch?v=O4eikzmaY8w"
# video_url = "https://www.youtube.com/watch?v=eYX_si-YSpg"
# video_url = "https://www.youtube.com/watch?v=YY0EOid9XRc"
# video_url = "https://www.youtube.com/watch?v=RMvnQEyQC4Y"
# video_url = "https://www.youtube.com/watch?v=Ogn5DcRIcbs"
# video_url = "https://www.youtube.com/watch?v=ZrBtJog3d9I"
# video_url = "https://www.youtube.com/watch?v=0oXEBCEwDQQ"
# video_url = "https://www.youtube.com/watch?v=gJlTHoB2Rj0"
# video_url = "https://www.youtube.com/watch?v=FvTCJMCpmGE"
# video_url = "https://www.youtube.com/watch?v=oX3wxl10qeo"
# video_url = "https://www.youtube.com/watch?v=raxwqCvDmwE"
# video_url = "https://www.youtube.com/watch?v=ISXdh2A8c-o"
video_url = "https://www.youtube.com/watch?v=zDE9UbiY7fg"
# video_url = ""




In [ ]:
# 下载并分割reviews


result = loadYoutubeData(video_url)#拉取 transcript
print (f'You have {len(result)} document(s) in your data')
print (f'There are {len(result[0].page_content)} characters in your document')

chunk_size= 1000#chunk大小 字符数
chunk_overlap = 100#chunk重叠 字符数
video_title = result[0].metadata['title'].replace(" ", "") #标题
video_date= result[0].metadata['publish_date'].strftime("%Y-%m-%d")#日期

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=100)#分割文档
texts = text_splitter.split_documents(result)#分割文档
print (f'There are {len(texts)} chunk split from your document')

#构建新数据
new_data=[{ 'id':str(uuid.uuid4()),'text': x.page_content.replace("\n", " "),'title':x.metadata['title'] , 'author': x.metadata['author'],'publish_date':video_date,'chunk_size':chunk_size,'chunk_overlap':chunk_overlap, 'thumbnail_url':x.metadata['thumbnail_url'],'url':video_url } for x in texts]

#储存新数据
json_str = json.dumps(new_data)
with open(f'data/{video_title}.json', 'w') as f:
    f.write(json_str)

print(new_data[0:2])

In [ ]:
# embedding 生成index并储存
batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(new_data), batch_size)):
    # find end of batch
    i_end = min(len(new_data), i+batch_size)
    meta_batch = new_data[i:i_end]
    # get ids
    ids_batch = [x['id'] for x in meta_batch]
    # get texts to encode
    texts = [x['text'] for x in meta_batch]
    # create embeddings (try-except added to avoid RateLimitError)
    # try:
    res = openai.Embedding.create(input=texts, engine=embed_model)
    # except:
    #     done = False
    #     while not done:
    #         sleep(5)
    #         try:
    #             res = openai.Embedding.create(input=texts, engine=embed_model)
    #             done = True
    #         except:
    #             pass
    embeds = [record['embedding'] for record in res['data']]
    # cleanup metadata
    meta_batch = [{
        'author': x['author'],
        'chunk_overlap': x['chunk_overlap'],
         'chunk_size': x['chunk_size'],
        'title': x['title'],
        'text': x['text'],
        'url': x['url'],
        'publish_date': x['publish_date'],
        'thumbnail_url': x['thumbnail_url']
    } for x in meta_batch]
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

# save index
index_str = json.dumps(to_upsert)
with open(f'index/{video_title}.json', 'w') as f:
    f.write(index_str)


In [ ]:
input("Press Enter to continue...")

In [65]:
query = "simulation game"
xq = openai.Embedding.create(input=query, engine=embed_model)['data'][0]['embedding']
res = index.query([xq], top_k=1000, include_metadata=True)

In [57]:
_txt = ""
for match in res['matches']:
    if 'title' not in match['metadata']:#因为数据不干净，有些没有title
       continue
    _txt +=  '\n\n*'+ '{:.3f}'.format(match['score'])+ '; [Name]: '+match['metadata']['title'] +'; [Descriptions]: '+ match['metadata']['text'] + ' [END]'
print(hash(_txt))
print(_txt)
with open('context/%s.txt' % hash(_txt), 'w', encoding='utf-8') as outfile:
    outfile.write(_txt)


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 5)

In [66]:
print(len(res['matches']))
for match in reversed(res['matches']):

    print(match['score'])


310
0.723892391
0.724246681
0.724345684
0.724401832
0.725319445
0.729561567
0.731738
0.732543111
0.732800126
0.73419863
0.737579823
0.739119887
0.739251792
0.740646482
0.743808448
0.744092464
0.745807588
0.746018
0.747059
0.747874498
0.749979258
0.750138283
0.75015825
0.750176132
0.750298619
0.750778437
0.750821114
0.750995278
0.75105381
0.751334965
0.752050102
0.752193928
0.75290966
0.753151059
0.753202319
0.7535429
0.753632784
0.754007339
0.754301
0.754484236
0.754634
0.754854
0.754987121
0.755496264
0.756017804
0.756183743
0.756934464
0.758143961
0.758910894
0.758978069
0.759197712
0.759513
0.759980321
0.760782659
0.761429906
0.761832774
0.76202178
0.762298048
0.762339711
0.762386143
0.762397647
0.762756586
0.763064086
0.763439238
0.763482332
0.763553083
0.763823807
0.76393348
0.764090121
0.764471233
0.764476061
0.764713109
0.764906645
0.765036821
0.765088558
0.765088558
0.765249789
0.765288532
0.765411198
0.76561439
0.76579529
0.76584214
0.765913844
0.765953124
0.765953124
0.766529